# Fraud Detection Baseline (Google Colab)

This notebook trains a low-memory baseline model for card transaction fraud detection using the `creditcard.csv` dataset.

Outputs:
- PR-AUC
- ROC-AUC
- Recall at fixed FPR (default 2%)
- Suggested thresholds for `approve / review / decline`
- Saved model artifact (`baseline_logreg.joblib`)

In [ ]:
!pip -q install numpy pandas scikit-learn matplotlib joblib

## 1) Load dataset
Use one of these options:
- Upload `creditcard.csv` directly from your machine
- Point `DATA_PATH` to a file in Google Drive

In [ ]:
from pathlib import Path

UPLOAD_FROM_BROWSER = True

if UPLOAD_FROM_BROWSER:
    try:
        from google.colab import files
    except ImportError:
        raise RuntimeError('Run this notebook in Google Colab or set UPLOAD_FROM_BROWSER=False and DATA_PATH manually.')

    uploaded = files.upload()
    if not uploaded:
        raise ValueError('No file uploaded. Upload creditcard.csv to continue.')
    DATA_PATH = next(iter(uploaded.keys()))
else:
    # Example: /content/drive/MyDrive/Fraud-ML/data/creditcard.csv
    DATA_PATH = '/content/drive/MyDrive/Fraud-ML/data/creditcard.csv'

print('DATA_PATH =', DATA_PATH)
assert Path(DATA_PATH).exists(), f'File not found: {DATA_PATH}'

## 2) Data contract + helper functions

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score, roc_curve
import joblib

LABEL_COLUMN = 'Class'
TIME_COLUMN = 'Time'
REQUIRED_COLUMNS = {TIME_COLUMN, 'Amount', LABEL_COLUMN, *{f'V{i}' for i in range(1, 29)}}
LEAKAGE_COLUMNS = {LABEL_COLUMN}

def validate_dataframe(df: pd.DataFrame, require_label: bool = True) -> None:
    required = set(REQUIRED_COLUMNS)
    if not require_label:
        required.discard(LABEL_COLUMN)
    missing = sorted(required - set(df.columns))
    if missing:
        raise ValueError(f'Missing required columns: {missing}')

def feature_columns(columns):
    return sorted([c for c in columns if c not in LEAKAGE_COLUMNS])

def recall_at_fpr(y_true: np.ndarray, y_score: np.ndarray, target_fpr: float = 0.02):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    valid = np.where(fpr <= target_fpr)[0]
    if len(valid) == 0:
        return 0.0, 1.0
    i = valid[-1]
    return float(tpr[i]), float(thresholds[i])

## 3) Read and prepare data (memory-efficient)

In [ ]:
df = pd.read_csv(DATA_PATH)
validate_dataframe(df, require_label=True)

float_cols = [c for c in df.columns if c != LABEL_COLUMN]
df[float_cols] = df[float_cols].astype(np.float32)
df[LABEL_COLUMN] = df[LABEL_COLUMN].astype(np.int8)

# Temporal split: train on past, test on future-like slice
df = df.sort_values(TIME_COLUMN)
split_idx = int(len(df) * 0.8)
train_df = df.iloc[:split_idx].copy()
test_df = df.iloc[split_idx:].copy()

X_cols = feature_columns(train_df.columns)
X_train = train_df[X_cols].to_numpy(dtype=np.float32)
y_train = train_df[LABEL_COLUMN].to_numpy(dtype=np.int8)
X_test = test_df[X_cols].to_numpy(dtype=np.float32)
y_test = test_df[LABEL_COLUMN].to_numpy(dtype=np.int8)

print('Train shape:', X_train.shape, 'Fraud rate:', y_train.mean())
print('Test shape :', X_test.shape, 'Fraud rate:', y_test.mean())

## 4) Train baseline model

In [ ]:
model = LogisticRegression(class_weight='balanced', max_iter=500, n_jobs=-1)
model.fit(X_train, y_train)

y_score = model.predict_proba(X_test)[:, 1]

## 5) Evaluate metrics and choose threshold

In [ ]:
TARGET_FPR = 0.02

pr_auc = average_precision_score(y_test, y_score)
roc_auc = roc_auc_score(y_test, y_score)
recall, threshold = recall_at_fpr(y_test, y_score, target_fpr=TARGET_FPR)

print('=== Baseline Metrics ===')
print(f'PR-AUC: {pr_auc:.6f}')
print(f'ROC-AUC: {roc_auc:.6f}')
print(f'Recall @ FPR <= {TARGET_FPR:.2%}: {recall:.6f}')
print(f'Threshold for target FPR: {threshold:.6f}')

In [ ]:
# Draft 3-band policy based on model score distribution
T1 = float(np.quantile(y_score, 0.90))  # review threshold
T2 = float(threshold)                   # decline threshold from FPR target

print('Suggested policy thresholds:')
print(f'approve: score < {T1:.6f}')
print(f'review : {T1:.6f} <= score < {T2:.6f}')
print(f'decline: score >= {T2:.6f}')

if T1 > T2:
    print('Note: T1 > T2 with this distribution. Set T1 manually below T2 after reviewing score histogram.')

## 6) Save artifact

In [ ]:
artifact = {
    'model': model,
    'feature_columns': X_cols,
    'threshold_for_target_fpr': float(threshold),
    'target_fpr': float(TARGET_FPR),
}
joblib.dump(artifact, 'baseline_logreg.joblib')
print('Saved: baseline_logreg.joblib')

## Next step
Integrate this artifact into a FastAPI `/predict` endpoint, then serve with Gunicorn + Uvicorn workers.